# Event Classification from EEG


**Introduction**
The following analyes EEG data taken in experiments where participants where exposed to light and sound events. This code cleans that 32 channel EEG data and uses a long short-term memory (LSTM) recurrent neural network to **classify the time following light or sound events by which event occured.**

### More settigs

**saved model details**
```python
standardized
model = Sequential()
model.add(Embedding(2, output_dim=256))
model.add(LSTM(100, input_shape=(time_steps, n_features))) model.add(Dropout(0.5)) model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=16, epochs=50) score = model.evaluate(X_test, y_test, batch_size=16)
```
This model run for 50 epochs had:
binary crossentropy 0.41922928811677918
accuracy 0.8529411764705882

**Take 2:: Add more epochs, increase batch size**


```python
model = Sequential()
model.add(LSTM(100, return_sequences=False, input_shape=(time_steps, n_features)))
model.add(Dropout(0.5))
#model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.fit(X_train, y_train, batch_size=32, epochs=500)
score = model.evaluate(X_test, y_test, batch_size=32)
```


In [1]:
# setting the random seed for reproducibility
import random 
seed = 42
random.seed(seed)

### Preparing the data 

In [2]:
# import all stuff
import itertools
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

### Loading the data 

In [3]:
# eeg1 and events1 are the test data from a single person
# code assumes eeg1 and events1 are csv files in the working dir
eeg1 = pd.read_csv("eeg1.csv", delimiter="\t")
new_cols = eeg1.columns.values
new_cols[0] = 'time'
new_cols[33] = 'sample'
eeg1.columns = new_cols

events1 = pd.read_csv("events1.csv")

In [4]:
# subsample of data to ease building the model, unused in final run
eeg1_smol = eeg1[0:785000]
events_smol = events1[0:1000]

In [5]:
# Toy data generator, unused in final run

def generate_eeg(samples, time_steps, n_features, event_types):
    # samples is Int number of trials 
    # time_steps is Int length of each trial in ms
    # n_features is Int number of EEG channels
    # event_types is Int number of stimula like lights and flashes
    signals = generate_signals(samples, time_steps, n_features)
    events = generate_events(event_types, samples)
    events_1hot = one_hot_events(events)
    return signals, events_1hot

# helper function (generate_eeg) for making EEG signal data
def generate_signals(samples, time_steps, n_features):
    # data types same as main function
    signals = np.random.random((samples, time_steps, n_features))
    return signals

# helper function (generate_eeg) for making one sample per event an
def generate_events(event_types, samples):
    # data types same as main function
    events = np.random.randint(1, event_types, samples)
    return events

In [6]:
# takes in eeg dataframe and event dataframe, cleans them, 
# 1hot encodes the events
def clean_eeg(eeg, events, event_interval_length, eeg_slice_length):
    #event_list = []
    array_list = [] 
    index_list = []
    eeg = standardize_eeg(eeg)
    """
    function for standardizing the eeg readings
    events_new = build_zero_events(events)
    iterate over the rows of the events and slice out the corresponding eeg data
    """
    for index, row in itertools.islice(events.iterrows(), event_interval_length): 
        # loop through events data
        #build_event_list(row, event_list) #
        tmin, tmax = build_event_intervals(row, events)
        eeg_slice = cut_event_intervals(eeg, tmin, tmax)
        array_list, index_list = build_array(eeg_slice, eeg_slice_length, 
                                             index, index_list, array_list)
    y_int = events.iloc[index_list] # take the event types for the correct index
    y_int = y_int['type'].values    # take just the event types as an array
    #y_int = y_int.as_matrix()            # save the event types as a matrix
    #y, lb = one_hot_events(y_int)        # one-hot the event types and save the binarizer
    X = np.stack(array_list, axis = 0)   # stack the arrays so the whole thing is 3D
    return X, y_int                     # return the data, outputs, and the binarizer
    
        
def build_event_list(row, event_list):
    # helper function to pull event types out of event data in the right order
    event_type = getattr(row, "type")
    event_list.append(event_type)
        
def build_event_intervals(row, events):
    # helper function to get the time intervals associated with each event
    tmin = getattr(row, "latency")
    tmin_in = getattr(row, "number")
    tmax_in = tmin_in + 1
    tmax = events1.loc[tmax_in, "latency"]
    return tmin, tmax

def cut_event_intervals(eeg, tmin, tmax):
    # helper function to slice up the eeg data so each slice is associated with one event
    eeg_slice = eeg.loc[(eeg["time"] > tmin) & (eeg["time"] < tmax)]
    eeg_slice.drop(["time", "sample"], axis = 1, inplace = True)
    return eeg_slice
    
def build_array(eeg_slice, eeg_slice_length, index, index_list, array_list):
    # helper function to build an array out of the eeg slices and 
    # pad them out to a standard length
    if len(eeg_slice) < eeg_slice_length:
        index_list.append(index)
        eeg_matrix = eeg_slice.as_matrix()
        padded_matrix = np.pad(eeg_matrix, ((0, eeg_slice_length - len(eeg_matrix)), (0,0)),
                                   'constant', constant_values=0)
        array_list.append(padded_matrix)
    return array_list, index_list

def one_hot_events(events):
    # helper function for one-hot encoding the events
    events_list = list(events)
    lb = preprocessing.LabelBinarizer()
    lb.fit(events_list)
    events_1hot = lb.transform(events_list)
    return events_1hot, lb

def invert_one_hot(events, lb):
    # function for decoding one-hot, binarizer made in one_hot_events
    inv_events = lb.inverse_transform(events)
    return inv_events

In [7]:
def standardize_eeg(eeg_data):
    # breaks apart an eeg dataframe, scales the eeg readings, and reassmbles it into a dataframe
    column_list = eeg_data.columns[1:33]
    time = eeg_data['time']
    sample = eeg_data['sample']
    eeg_array = eeg_data[column_list]
    eeg_stnd = scale_data(eeg_array)
    eeg_stnd_df = pd.DataFrame(eeg_stnd, index=eeg_data.index, columns=column_list)
    eeg_stnd = pd.concat([time, eeg_stnd_df, sample], axis =1)
    return eeg_stnd

def scale_data(unscaled_data):
    # helper function for standardize_eeg, fits a scaler and transforms the data 
    scaler = StandardScaler()
    scaler.fit(unscaled_data)
    scaled_data = scaler.transform(unscaled_data)
    return scaled_data

In [8]:
# This is unused code for breaking up the "nothing happened" periods of the eeg data 
# to associate with "type 0" events. 

import math
time_steps = 1300

def build_zero_events(event_data, time_steps=time_steps):
    new_events = build_new_events(event_data, time_steps)
    events = zero_events(event_data, new_events)
    return events


def build_new_events(event_data, time_steps= time_steps):
    first_event_time = event_data['latency'].loc[1]
    number_new_intervals = math.floor(first_event_time / time_steps)
    df = pd.DataFrame(columns=['number', 'latency', 'type', 'duration'],index = range(number_new_intervals) )
    latency = 0
    for t in range(number_new_intervals):
        latency += 1300
        df.loc[t].latency = latency
        df.loc[t].type = 0
    return df

def zero_events(event_data, new_events):
    events_zeros = event_data[event_data.latency != 1]
    events_zeros= new_events.append(events_zeros)
    events_zeros = events_zeros.reset_index(drop=True)
    events_zeros['number'] = events_zeros.index + 1
    return events_zeros

## Building the Model 

In [1]:
# full dataset parameters

# define model parameters

# number trials of eeg data
sample = 3600
# number of channels of eeg in exch sample
n_features = 32
# number of steps(ms) each sample was run for
time_steps = 1300
# event types , number of different types of events (light, sound, other?)
event_types = 2

In [10]:
# get the data into useable form and store as X and y
X, y = clean_eeg(eeg1, events1, samples, time_steps)  
#4250 long, 998 short, 4330 long enhanced

/home/deeplearn/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [11]:
# removes the minor event types. There were only a couple hundred examples of each, whereas the used events had a 
# couple thousand examples
remove_list = [0,2,4,5,6]              # designate unwanted event types
drop_list = np.isin(y, remove_list)    # create a list of indices associated with unwanted events                  
drop_array = np.array(drop_list)       # make the list of indices to drop into an array

In [12]:
# make X, y's with unwanted events removed
y_short_int = y[np.isin(y, remove_list, invert=True)]
X_short     = X[np.isin(y, remove_list, invert=True)]

In [13]:
# one-hot encode the y data without the unwanted events
y_short, lb = one_hot_events(y_short_int)

In [14]:
from sklearn.model_selection import StratifiedShuffleSplit

# use strat.shuffle.split to get indices for test and training data
sss = StratifiedShuffleSplit(n_splits = 2, test_size = 0.2, random_state = seed)
sss.get_n_splits(X_short, y_short)

2

In [15]:
# take the indices generated by stratified shuffle split and make the test and training datasets
for train_index, test_index in sss.split(X_short, y_short):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X_short[train_index], X_short[test_index]
    y_train, y_test = y_short[train_index], y_short[test_index]

In [17]:
# import required LSTM stuff from keras

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import LSTM
# build LSTM model with 100 neurons and dropout etc

model = models.Sequential()
model.add(LSTM(100, return_sequences = False, input_shape = (time_steps, n_features)))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile our model

model.compile(loss = 'binary_crossentropy' ,
             optimizer = 'rmsprop',
             metrics = ['accuracy'])

# run & evaluate the model

model.fit(X_train, y_train, batch_size = 32, epochs = 50)
score = model.evaluate(X_test, y_test, batch_size = 32)

Epoch 1/500
2444/2444 [==============================] - 50s - loss: 0.6788 - acc: 0.5953    
Epoch 2/500
2444/2444 [==============================] - 50s - loss: 0.6769 - acc: 0.5978    
Epoch 3/500
2444/2444 [==============================] - 50s - loss: 0.6763 - acc: 0.5978    
Epoch 4/500
2444/2444 [==============================] - 50s - loss: 0.6736 - acc: 0.5949    
Epoch 5/500
2444/2444 [==============================] - 50s - loss: 0.6722 - acc: 0.5970    
Epoch 6/500
2444/2444 [==============================] - 50s - loss: 0.6692 - acc: 0.5962    
Epoch 7/500
2444/2444 [==============================] - 50s - loss: 0.6649 - acc: 0.6007    
Epoch 8/500
2444/2444 [==============================] - 50s - loss: 0.6593 - acc: 0.6150    
Epoch 9/500
2444/2444 [==============================] - 50s - loss: 0.6418 - acc: 0.6412    
Epoch 10/500
2444/2444 [==============================] - 50s - loss: 0.6391 - acc: 0.6551    
Epoch 11/500
2444/2444 [==============================] - 5

2444/2444 [==============================] - 50s - loss: 0.0390 - acc: 0.9910    
Epoch 173/500
2444/2444 [==============================] - 50s - loss: 0.0431 - acc: 0.9906    
Epoch 174/500
2444/2444 [==============================] - 50s - loss: 0.0396 - acc: 0.9910    
Epoch 175/500
2444/2444 [==============================] - 49s - loss: 0.0329 - acc: 0.9914    
Epoch 176/500
2444/2444 [==============================] - 50s - loss: 0.0379 - acc: 0.9935    
Epoch 177/500
2444/2444 [==============================] - 50s - loss: 0.0292 - acc: 0.9922    
Epoch 178/500
2444/2444 [==============================] - 50s - loss: 0.0468 - acc: 0.9906    
Epoch 179/500
2444/2444 [==============================] - 50s - loss: 0.0279 - acc: 0.9935    
Epoch 180/500
2444/2444 [==============================] - 50s - loss: 0.0401 - acc: 0.9902    
Epoch 181/500
2444/2444 [==============================] - 50s - loss: 0.0611 - acc: 0.9865    
Epoch 182/500
2444/2444 [=============================

2444/2444 [==============================] - 50s - loss: 0.0316 - acc: 0.9943    
Epoch 343/500
2444/2444 [==============================] - 50s - loss: 0.0193 - acc: 0.9959    
Epoch 344/500
2444/2444 [==============================] - 50s - loss: 0.0199 - acc: 0.9959    
Epoch 345/500
2444/2444 [==============================] - 50s - loss: 0.0355 - acc: 0.9939    
Epoch 346/500
2444/2444 [==============================] - 50s - loss: 0.0430 - acc: 0.9902    
Epoch 347/500
2444/2444 [==============================] - 49s - loss: 0.0250 - acc: 0.9943    
Epoch 348/500
2444/2444 [==============================] - 50s - loss: 0.0189 - acc: 0.9971    
Epoch 349/500
2444/2444 [==============================] - 50s - loss: 0.0068 - acc: 0.9988    
Epoch 350/500
2444/2444 [==============================] - 50s - loss: 0.0184 - acc: 0.9959    
Epoch 351/500
2444/2444 [==============================] - 49s - loss: 0.0174 - acc: 0.9967    
Epoch 352/500
2444/2444 [=============================

In [18]:
score

[0.9323037510387379, 0.9150326797385621]

In [19]:
print("Accuracy: %.2f%%" % (score[1]*100))

Accuracy: 91.50%
